# Base Model for Demand Forecasting

The main objective is to optimize inventory and purchasing management, with a target of **reducing overstocking by 20%** within 6 months.

- Target Variable for Inventory Optimization: **Stock_Quantity**
- Target Variable for Demand Forecasting: **Sales_Volume**

### Metrics for models avaliation
- RMSE
- MAE

# DATA ACQUISITION
## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings('ignore')

## Load Data

In [2]:
# Define data paths
data_path = os.path.join('../data', 'processed')

In [3]:
df = pd.read_pickle(data_path + '/grocery.pkl')

# Feature Enginnering

In [4]:
# Applying differentiation in non stationary variables
df['Delivery_Lag_diff'] = df['Delivery_Lag'].diff().fillna(0)
df['Days_For_Expiration_diff'] = df['Days_For_Expiration'].diff().fillna(0)

In [5]:
# Ascending date
df = df.sort_values(by='Date_Received').reset_index(drop=True)

In [6]:
# Index by Date_Received, dialy frequency
df_ts = df.set_index('Date_Received')

In [24]:
pd.to_pickle(df_ts, data_path + '/time_serie.pkl')

## Split data

In [7]:
# Target Columns
y = df_ts[['Sales_Volume', 'Stock_Quantity']]

In [8]:
# For modeling, we removed highly correlated columns and unique identifiers that did not add predictive value.
drop_columns = ['Product_ID', 'Supplier_ID', 'Last_Order_Date', 'Expiration_Date',
       'Warehouse_Location', 'Stock_Value', 'Days_For_Expiration', 
       'Purchase_Order', 'Delivery_Lag'] + y.columns.to_list()

In [9]:
# Feature Columns
X= df_ts.drop(columns=drop_columns)

In [10]:
# Split data in train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

## Encode Features

### One Hot

In [11]:
# Encode Non numeric Variables
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [12]:
# Columns for One Hot
columns_ = ['Category', 'Status', 'Expiration_Status']

# Fit in X_train
encoder.fit(X_train[columns_])

# Transform X 
X_train_encoded = encoder.transform(X_train[columns_])
X_test_encoded = encoder.transform(X_test[columns_])

In [13]:
# Create encoded dataframes
encoded_columns_name = encoder.get_feature_names_out(columns_)
X_train_encoded = pd.DataFrame(X_train_encoded, columns=encoded_columns_name, index=X_train.index)
X_test_encoded = pd.DataFrame(X_test_encoded, columns=encoded_columns_name, index=X_test.index)

In [14]:
# Union datasets
X_train = pd.concat([X_train.drop(columns=columns_), X_train_encoded], axis=1)
X_test = pd.concat([X_test.drop(columns=columns_), X_test_encoded], axis=1)

### Feature Extraction

In [15]:
# Combine the columns in list of list
X_train_to_hash = [[prod, supp] for prod, supp in zip(X_train['Product_Name'], X_train['Supplier_Name'])]
X_test_to_hash = [[prod, supp] for prod, supp in zip(X_test['Product_Name'], X_test['Supplier_Name'])]

In [16]:
# Create Feature Hashing
n_features=100
hasher = FeatureHasher(n_features=n_features, input_type="string")

In [17]:
# Appling
X_train_hashed = hasher.transform(X_train_to_hash)
X_test_hashed = hasher.transform(X_test_to_hash)

In [18]:
# Columns Names for data
hashed_column_names = [f'hashed_feature_{i}' for i in range(n_features)]

In [19]:
# Create DataFrame with dense matrix
X_train_hashed_df = pd.DataFrame(X_train_hashed.toarray(), columns=hashed_column_names, index=X_train.index)
X_test_hashed_df = pd.DataFrame(X_test_hashed.toarray(), columns=hashed_column_names, index=X_test.index)


In [20]:
# Create Final DataFrame
# Columns to remove
columns_rm = X_train[['Product_Name', 'Supplier_Name']].columns.to_list()

# Concatenate wit others DataFrames
X_train_final = pd.concat([
    X_train.drop(columns=columns_rm),
    X_train_hashed_df
], axis=1)

X_test_final = pd.concat([
    X_test.drop(columns=columns_rm),
    X_test_hashed_df
], axis=1)

In [21]:
X_test_final

,Reorder_Level,Reorder_Quantity,Unit_Price,Inventory_Turnover_Rate,Stock_Coverage_Days,Delivery_Lag_diff,Days_For_Expiration_diff,Category_Bakery,Category_Beverages,Category_Dairy,...,hashed_feature_90,hashed_feature_91,hashed_feature_92,hashed_feature_93,hashed_feature_94,hashed_feature_95,hashed_feature_96,hashed_feature_97,hashed_feature_98,hashed_feature_99
Date_Received,,,,,,,,,,,,,,,,,,,,,
2024-12-11,90,57,25.00,68,5,40.0,-178,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-12-12,68,90,2.00,13,28,175.0,-286,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-12-12,69,90,4.00,35,10,172.0,-12,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-12-12,36,96,6.50,73,5,122.0,-242,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-12-12,94,18,6.00,11,33,-29.0,-134,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-23,69,64,2.75,6,60,351.0,-84,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-02-23,61,19,2.00,69,5,25.0,115,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-02-24,40,29,15.00,87,4,59.0,-392,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
X_train_final

,Reorder_Level,Reorder_Quantity,Unit_Price,Inventory_Turnover_Rate,Stock_Coverage_Days,Delivery_Lag_diff,Days_For_Expiration_diff,Category_Bakery,Category_Beverages,Category_Dairy,...,hashed_feature_90,hashed_feature_91,hashed_feature_92,hashed_feature_93,hashed_feature_94,hashed_feature_95,hashed_feature_96,hashed_feature_97,hashed_feature_98,hashed_feature_99
Date_Received,,,,,,,,,,,,,,,,,,,,,
2024-02-25,89,25,21.0,62,5,0.0,-18,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-02-26,29,76,2.4,4,91,0.0,292,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-02-26,90,77,4.0,19,19,-66.0,80,0.0,0.0,0.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-02-26,96,85,2.5,3,121,0.0,247,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-02-27,60,21,8.9,46,7,0.0,33,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-10,93,61,6.0,100,3,-93.0,-11,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-12-10,12,85,5.0,46,7,72.0,52,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-12-10,98,97,6.0,21,17,-100.0,-278,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Multivariate Model

In [ ]:
# Features (lags)
X = df.drop(columns=["Date", "Sales_Volume", "Stock_Quantity"])
y = df[["Sales_Volume", "Stock_Quantity"]]

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Modelo multi-output
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
model.fit(X_train, y_train)

preds = model.predict(X_test)